In [1]:
"""
Zero-curve bootstrap → Fed-style table → interactive visuals
"""

# ────────────────────────────────────────────────────────────────
# 1. Imports
# ────────────────────────────────────────────────────────────────
import numpy as np
import pandas as pd
from pathlib import Path

# finance libs (install / adjust as required)
from calendars.daycounts import DayCounts
from finmath.termstructure.curve_models import CurveBootstrap
from finmath.brazilian_bonds.corporate_bonds import CorpsCalcs1

# optional: SciPy interpolation (fallback to numpy.interp if absent)
try:
    from scipy.interpolate import interp1d
except ImportError:
    interp1d = None
    print("SciPy not found – falling back to numpy.interp (no extrapolation)")

# Plotly for visuals
import plotly.express as px
import plotly.graph_objects as go

# ────────────────────────────────────────────────────────────────
# 1. Globals – audit collector
# ────────────────────────────────────────────────────────────────
AUDIT_LOG: list[pd.DataFrame] = []  # filled inside build_zero_curve

# ────────────────────────────────────────────────────────────────
# 2. Locate repo root & data files
# ────────────────────────────────────────────────────────────────
REPO_ROOT = Path.cwd()
while not (REPO_ROOT / ".git").exists() and REPO_ROOT != REPO_ROOT.parent:
    REPO_ROOT = REPO_ROOT.parent

CORP_PATH = REPO_ROOT / "datos_y_modelos" / "Domestic" / "brazil_domestic_corp_db.xlsx"
DI_PATH = REPO_ROOT / "datos_y_modelos" / "db" /  "one-day_interbank_deposit_futures_contract_di" / "bsrch.xlsx"
YIELD_PATH = Path("../db/one-day_interbank_deposit_futures_contract_di/ODA_Comdty.xlsx")
CORP_YIELD_PATH = Path("../db/brazil_domestic_corp_bonds/ya.xlsx")


ID_COL = "id"     # change here if necessary
DAYCOUNT = DayCounts("bus/252", calendar="cdr_anbima")

# ────────────────────────────────────────────────────────────────
# 3. Static bond characteristics
# ────────────────────────────────────────────────────────────────
bonds_static = pd.read_excel(DI_PATH, sheet_name="periods_values_only")
bonds_static["End of Month date"]     = pd.to_datetime(bonds_static["End of Month date"])
bonds_static["Settlement date"]     = pd.to_datetime(bonds_static["Settlement date"])
bonds_static




# ────────────────────────────────────────────────────────────────
# 4. Historical YAS yield matrix
# ────────────────────────────────────────────────────────────────
ylds = pd.read_excel(YIELD_PATH, sheet_name="ya_values_only")
ylds.rename(columns={ylds.columns[0]: "OBS_DATE"}, inplace=True)
ylds["OBS_DATE"] = pd.to_datetime(ylds["OBS_DATE"])
ylds = ylds.set_index("OBS_DATE").sort_index()
ylds




,BZDIOVRA Index,od1 Comdty,od2 Comdty,od3 Comdty,od4 Comdty,od5 Comdty,od6 Comdty,od7 Comdty,od8 Comdty,od9 Comdty,...,od31 Comdty,od32 Comdty,od33 Comdty,od34 Comdty,od35 Comdty,od36 Comdty,od37 Comdty,od38 Comdty,od39 Comdty,od40 Comdty
OBS_DATE,,,,,,,,,,,,,,,,,,,,,
1993-02-26,0.00,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1993-03-31,0.00,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1993-04-30,0.00,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1993-05-31,0.00,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
1993-06-30,0.00,0.00,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-28,13.15,13.15,13.553,13.876,14.150,14.336,14.504,14.635,14.752,14.834,...,15.107,15.124,0.000,15.142,0.000,15.154,15.169,15.127,15.072,15.050
2025-03-31,14.15,14.15,14.156,14.391,14.534,14.683,14.791,14.873,14.928,14.969,...,14.789,14.807,14.824,0.000,14.848,14.881,14.866,14.824,14.827,14.783
2025-04-30,14.15,14.15,14.469,14.529,14.598,14.633,14.658,14.664,14.663,14.659,...,13.692,13.740,0.000,13.778,13.841,13.853,13.836,13.845,13.844,13.828


In [ ]:

# ────────────────────────────────────────────────────────────────
# 3. Build surface: obs_date, tenor (yrs), yield (%)
# ────────────────────────────────────────────────────────────────
long = (
    ylds.reset_index()
        .melt(id_vars="OBS_DATE",
              var_name="Generic ticker",
              value_name="yield")
        .dropna(subset=["yield"])
        .loc[lambda df: df["yield"] > 0]        # keep positives without .query
)

# merge with calendar and compute tenor – same as before
calendar_cols = ["Generic ticker", "Curve date", "End of Month days"]
bonds_key = bonds_static[calendar_cols].rename(columns={"Curve date": "OBS_DATE"})

merged = (long.merge(bonds_key, on=["Generic ticker", "OBS_DATE"], how="left").dropna(subset=["End of Month days"]))

surface = merged.rename(columns={"OBS_DATE": "obs_date", "Generic ticker": "id", "End of Month days": "tenor" })[["obs_date", "id", "yield", "tenor"]].reset_index(drop=True)

surface

,obs_date,id,yield,tenor
0,1994-07-29,od1 Comdty,59.059,23.0
1,1994-08-31,od1 Comdty,56.273,21.0
2,1994-09-30,od1 Comdty,62.715,18.0
3,1994-10-31,od1 Comdty,66.876,20.0
4,1994-11-30,od1 Comdty,58.258,22.0
...,...,...,...,...
6560,2025-02-28,od40 Comdty,15.050,832.0
6561,2025-03-31,od40 Comdty,14.783,836.0
6562,2025-04-30,od40 Comdty,13.828,838.0
6563,2025-05-30,od40 Comdty,13.690,838.0


In [ ]:

# ────────────────────────────────────────────────────────────────
# 4. Interpolate to Fed-style tenors
# ────────────────────────────────────────────────────────────────
TENORS = {
    "30-year": 30.0, "10-year": 10.0, "5-year": 5.0, "3-year": 3.0,
    "2-year":  2.0,  "1-year":  1.0, "6-month": 0.5,
    "3-month": 0.25, "1-month": 1/12,
}

rows = []
for obs_date, grp in surface.groupby("obs_date"):
    grp = grp.drop_duplicates(subset="tenor").sort_values("tenor")
    if grp["tenor"].nunique() < 2:
        continue                                         # need ≥2 pts for interp

    x = grp["tenor"].to_numpy()
    y = grp["yield"].to_numpy()

    if len(x) > 1:
        f = interp1d(x, y, kind="linear",
                     bounds_error=False, fill_value="extrapolate",
                     assume_sorted=True)
        interp = lambda t: float(f(t))
    else:                                               # single-point fallback
        interp = lambda t: float(y[0])

    rows.append({"DATE": obs_date.date(),
                 **{k: interp(t) for k, t in TENORS.items()}})

yc_table = (
    pd.DataFrame(rows)
      .set_index("DATE")
      .sort_index()
      .dropna(how="any")       # guarantees clean for plotting
)


yc_table

,30-year,10-year,5-year,3-year,2-year,1-year,6-month,3-month,1-month
DATE,,,,,,,,,
1994-07-29,55.574000,65.531143,68.020429,69.016143,69.514000,70.011857,70.260786,70.385250,70.468226
1994-08-31,55.975526,56.636579,56.801842,56.867947,56.901000,56.934053,56.950579,56.958842,56.964351
1994-09-30,63.375000,62.275000,62.000000,61.890000,61.835000,61.780000,61.752500,61.738750,61.729583
1994-10-31,67.335091,66.416909,66.187364,66.095545,66.049636,66.003727,65.980773,65.969295,65.961644
1994-11-30,58.805810,57.436286,57.093905,56.956952,56.888476,56.820000,56.785762,56.768643,56.757230
...,...,...,...,...,...,...,...,...,...
2025-02-28,13.404526,12.980316,12.874263,12.831842,12.810632,12.789421,12.778816,12.773513,12.769978
2025-03-31,14.152857,14.147143,14.145714,14.145143,14.144857,14.144571,14.144429,14.144357,14.144310
2025-04-30,14.293550,13.974550,13.894800,13.862900,13.846950,13.831000,13.823025,13.819038,13.816379


In [ ]:


# ────────────────────────────────────────────────────────────────
# 8. Plotting helpers
# ────────────────────────────────────────────────────────────────
def plot_yield_curve_surface(df, source_text=""):
    short_col = df.columns[0]             # line on shortest tenor
    zmin, zmax = df.values.min(), df.values.max()
    fig = go.Figure()
    fig.add_trace(go.Surface(
        x=df.columns, y=df.index, z=df.values,
        colorscale="ice", reversescale=True,
        cmin=zmin, cmax=zmax,
        hovertemplate="<br>Date: %{y}"
                      "<br>Maturity: %{x}"
                      "<br>Yield: %{z:.2f}%<extra></extra>"
    ))
    fig.add_trace(go.Scatter3d(
        x=[short_col]*len(df), y=df.index, z=df[short_col],
        mode="lines", line=dict(color="black", width=1.5),
        name=f"{short_col} yield"
    ))
    fig.update_layout(
        title="3-D Yield-Curve Surface", height=900,
        scene=dict(
            aspectratio=dict(x=1, y=1.75, z=0.75),
            camera=dict(eye=dict(x=1.65, y=1.57, z=0.25))
        ),
        margin=dict(l=0, r=0, t=40, b=10),
        annotations=[dict(text=source_text, x=0, y=0.02,
                          xref="paper", yref="paper", showarrow=False)]
    )
    return fig

def plot_audit_3d(surface_df: pd.DataFrame) -> go.Figure:
    """
    3-D scatter for DI futures:
        x = tenor (years, numeric),
        y = obs_date,
        z = yield (%).
    """
    pts = surface_df.sort_values("obs_date")

    fig = go.Figure(data=[
        go.Scatter3d(
            x=pts["tenor"],               # numeric tenor
            y=pts["obs_date"],
            z=pts["yield"],
            mode="markers",
            marker=dict(size=4, color="royalblue", opacity=0.8),
            text=(pts["id"] + "<br>" +
                  "Tenor: " + pts["tenor"].round(2).astype(str) + " yrs"),
            hovertemplate="<b>%{text}</b><br>Date: %{y|%Y-%m-%d}"
                          "<br>Yield: %{z:.2f}%<extra></extra>"
        )
    ])

    fig.update_layout(
        title="DI Futures – Yield vs. Tenor & Date",
        height=700,
        scene=dict(
            xaxis_title="Tenor (years)",
            yaxis_title="Obs date",
            zaxis_title="Yield (%)"
        ),
        margin=dict(l=20, r=20, t=40, b=40)
    )
    return fig
    

def plot_heatmap(df, source_text=""):
    zmin, zmax = df.values.min(), df.values.max()
    data = df.T.iloc[::-1]   # flip so shortest at bottom
    fig = go.Figure(data=[go.Heatmap(
        z=data.values, x=data.columns, y=data.index,
        colorscale="ice", reversescale=True,
        zmin=zmin, zmax=zmax,
        hovertemplate="<br>Date: %{x}"
                      "<br>Maturity: %{y}"
                      "<br>Yield: %{z:.2f}%<extra></extra>"
    )])
    fig.update_layout(
        title="Yield-Curve Heat-map", height=600,
        margin=dict(t=70, b=90, l=20, r=20),
        annotations=[dict(text=source_text, x=0, y=-0.15,
                          xref="paper", yref="paper", showarrow=False)]
    )
    return fig


def plot_historical_yield_curve(df, source_text="", id_vars="DATE"):
    df_rev = df.iloc[:, ::-1]
    melt = pd.melt(df_rev.reset_index(), id_vars=id_vars,
                   var_name="Maturity", value_name="Yield")
    melt[id_vars] = pd.to_datetime(melt[id_vars]).dt.strftime("%b-%Y")

    fig = px.line(
        melt, x="Maturity", y="Yield",
        animation_frame=id_vars, animation_group="Maturity",
        range_y=[df.values.min(), df.values.max()],
        color_discrete_sequence=["cornflowerblue"],
        labels={"Yield": "Yield (%)"}
    )
    latest = df_rev.iloc[-1]
    fig.add_trace(go.Scatter(x=latest.index, y=latest.values,
                             name="Latest", line=dict(color="black", width=3)))
    for s in fig.layout.sliders[0].steps:
        s["args"][1]["frame"]["redraw"] = True
    for b in fig.layout.updatemenus[0].buttons:
        b["args"][1]["frame"]["redraw"] = True
        b["args"][1]["frame"]["duration"] = 200
    fig.update_layout(
        title="Animated Yield-Curve History", height=600,
        margin=dict(t=70, b=90, l=20, r=20),
        annotations=[dict(text=source_text, x=0, y=-0.15,
                          xref="paper", yref="paper", showarrow=False)]
    )
    return fig


def plot_line_spread(df, low="2-year", high="10-year",
                     idx_name="DATE", source_text=""):
    data = df.copy()
    data["Spread"] = data[high] - data[low]
    mask = data["Spread"] <= 0
    data["Spread_above"] = np.where(mask, data["Spread"], 0)
    data["Spread_below"] = np.where(mask, 0, data["Spread"])
    fig = px.area(data.reset_index(), x=idx_name, y="Spread",
                  hover_data={"Spread": True},
                  labels={"Spread": f"{high} − {low} (%)"})
    fig.add_trace(go.Scatter(x=data.index, y=data["Spread_above"],
                             fill="tozeroy", mode="none"))
    fig.add_trace(go.Scatter(x=data.index, y=data["Spread_below"],
                             fill="tozeroy", mode="none"))
    fig.update_layout(
        title=f"Yield-Spread: {high} minus {low}", height=550,
        margin=dict(t=60, b=90, l=20, r=20),
        annotations=[dict(text=source_text, x=0, y=-0.15,
                          xref="paper", yref="paper", showarrow=False)],
        showlegend=False
    )
    return fig

# ────────────────────────────────────────────────────────────────
# 9. Visualise
# ────────────────────────────────────────────────────────────────
if __name__ == "__main__":
    # put columns shortest → longest for nicer surfaces
    ordered_cols = ["1-month", "3-month", "6-month",
                    "1-year", "2-year", "3-year",
                    "5-year", "10-year", "30-year"]
    df_vis = yc_table[ordered_cols]

    SOURCE = ("Source:DI B3 "
              "– author calculations")

    surface_fig   = plot_yield_curve_surface(df_vis, SOURCE)
    heatmap_fig   = plot_heatmap(df_vis, SOURCE)
    history_fig   = plot_historical_yield_curve(df_vis, SOURCE)
    spread_fig    = plot_line_spread(df_vis, low="2-year", high="10-year",
                                     source_text=SOURCE)

    #print
    
    # show (inline in Jupyter or pop-up in a script)
    surface_fig.show()

    audit_fig = plot_audit_3d(surface)
    audit_fig.show()

    
    heatmap_fig.show()
    history_fig.show()
    spread_fig.show()


In [5]:
# ────────────────────────────────────────────────────────────────
# 3. Static bond characteristics
# ────────────────────────────────────────────────────────────────
bonds_static = pd.read_excel(CORP_PATH, sheet_name="db_values_only")

wanted = [
    "BRRDORDBS042",
    "BREGIEDBS0E7",
    "BRENATDBS044",
]

corp_bonds = bonds_static.loc[bonds_static["ID_ISIN"].isin(wanted)].copy()

corp_bonds

,ID,id,ISSUER,ID_BB_GLOBAL,ID_ISIN,ISSUE_DT,MATURITY,CRNCY,CPN_TYP,CPN,...,MARKET_ISSUE,CNTRY_OF_RISK,FIRST_CPN_DT,MTY_TYP,AMT_ISSUED,AMT_OUTSTANDING,MARKET_OF_DISTRIBUTION,TOT_DEBT_TO_EBITDA,EXCH_CODE,CIE DES BULK
247,BI018411 Corp,BI018411 Corp,REDE D'OR SAO LUIZ SA,BBG00TCNBSY1,BRRDORDBS042,4/17/2020,1/17/2030,BRL,FIXED,8.7486,...,DOMESTIC,BR,7/20/2020,AT MATURITY,3198350000,3.198350e+09,Domestic,4.706104,BZ,Rede D'Or Sao Luiz S.A. provides medical and s...
2579,ZG625748 Corp,ZG625748 Corp,ENGIE BRASIL ENERGIA SA,BBG01KH2VYK0,BREGIEDBS0E7,11/15/2023,11/15/2028,BRL,FIXED,10.9000,...,DOMESTIC,BR,11/15/2028,AT MATURITY,318122000,3.181220e+08,Domestic,4.037184,FP,Engie provides low-carbon energy and services....
2669,ZH253136 Corp,ZH253136 Corp,ENAUTA PARTICIPACOES,BBG01JLW31Z9,BRENATDBS044,9/15/2023,9/17/2029,BRL,FIXED,13.9662,...,DOMESTIC,BR,3/15/2024,AT MATURITY,996504000,9.965040e+08,Domestic,6.396052,BZ,Brava Energia operates as a oil and gas compan...


In [6]:
# ────────────────────────────────────────────────────────────────
# 4. Historical YAS yield matrix
# ────────────────────────────────────────────────────────────────
corp_ylds = pd.read_excel(CORP_YIELD_PATH, sheet_name="ya_values_only")
corp_ylds.rename(columns={corp_ylds.columns[0]: "OBS_DATE"}, inplace=True)
corp_ylds["OBS_DATE"] = pd.to_datetime(corp_ylds["OBS_DATE"])
corp_ylds = corp_ylds.set_index("OBS_DATE").sort_index()
corp_ylds

,EC744185 Corp,BV932351 Corp,ZN887990 Corp,BX122675 Corp,ZD181681 Corp,ZJ024663 Corp,YN794038 Corp,BV905494 Corp,BQ182881 Corp,YX077422 Corp,...,EJ745835 Corp,ZG074649 Corp,EK062024 Corp,YT660642 Corp,ZD833929 Corp,ZM429069 Corp,BR853537 Corp,BT353913 Corp,BZ862695 Corp,YT877069 Corp
OBS_DATE,,,,,,,,,,,,,,,,,,,,,
1989-10-31,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0,0.0,0.000000,0.000000,0.000000,0
1989-11-30,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0,0.0,0.000000,0.000000,0.000000,0
1989-12-29,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0,0.0,0.000000,0.000000,0.000000,0
1990-01-31,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0,0.0,0.000000,0.000000,0.000000,0
1990-02-28,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0,0.0,0.000000,0.000000,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-28,NaN,-6666.000000,3.148550,3.159477,1.076930,1.306744,0,32.730055,1.255730,0.597712,...,-6666.000000,5.998689,0.000000,12.719988,0,NaN,-7.308758,0.443472,-2.670922,0
2025-03-31,NaN,926.058368,3.103330,3.532190,1.153315,1.294116,0,32.786838,1.172592,2.407520,...,88403.659687,6.162208,0.000000,13.132672,0,NaN,-7.908430,0.324448,-2.464705,0
2025-04-30,NaN,1211.869713,3.580347,3.784295,1.147126,1.306476,0,33.912204,1.192853,2.406513,...,163787.988422,5.615661,-14.691093,12.601764,0,NaN,-8.407286,0.274909,-2.552571,0


In [7]:


# ────────────────────────────────────────────────────────────────
# 2. Locate repo root & data files
# ────────────────────────────────────────────────────────────────

#SUPRA_PATH = REPO_ROOT / "datos_y_modelos" / "Domestic" / "supra.xlsx"
#CORP_PATH = REPO_ROOT / "datos_y_modelos" / "Domestic" / "brazil_domestic_corp_db.xlsx"
SUPRA_PATH = REPO_ROOT / "datos_y_modelos" / "Domestic" / "brazil_domestic_corp_db.xlsx"

#CORP_YIELD_PATH = Path("../db/brazil_domestic_corp_bonds/ya.xlsx")
#YIELD_PATH = Path("../db/supranational_bonds/ya.xlsx")
YIELD_PATH = Path("../db/brazil_domestic_corp_bonds/ya.xlsx")

# ────────────────────────────────────────────────────────────────
# 3. Static bond characteristics
# ────────────────────────────────────────────────────────────────
bonds_static = pd.read_excel(SUPRA_PATH, sheet_name="db_values_only")


#***********FILTRO***************************************
wanted = [
    "BRRDORDBS042",
    "BREGIEDBS0E7",
    "BRENATDBS044",
]

bonds_static = bonds_static.loc[bonds_static["ID_ISIN"].isin(wanted)].copy()
#***********FILTRO***************************************


# Converte a coluna de maturidade, forçando erro para virar NaT
bonds_static["MATURITY"] = pd.to_datetime(bonds_static["MATURITY"], errors='coerce')

bonds_static["FIRST_CPN_DT"] = pd.to_datetime(
    bonds_static["FIRST_CPN_DT"], errors="coerce"
)
bonds_static = (
    bonds_static
      .dropna(subset=["MATURITY"])
      .loc[bonds_static["MATURITY"] >= "2000-01-01"]
      .set_index(ID_COL)
)

bonds_static

,ID,ISSUER,ID_BB_GLOBAL,ID_ISIN,ISSUE_DT,MATURITY,CRNCY,CPN_TYP,CPN,FLT_SPREAD,...,MARKET_ISSUE,CNTRY_OF_RISK,FIRST_CPN_DT,MTY_TYP,AMT_ISSUED,AMT_OUTSTANDING,MARKET_OF_DISTRIBUTION,TOT_DEBT_TO_EBITDA,EXCH_CODE,CIE DES BULK
id,,,,,,,,,,,,,,,,,,,,,
BI018411 Corp,BI018411 Corp,REDE D'OR SAO LUIZ SA,BBG00TCNBSY1,BRRDORDBS042,4/17/2020,2030-01-17,BRL,FIXED,8.7486,#N/A Field Not Applicable,...,DOMESTIC,BR,2020-07-20,AT MATURITY,3198350000,3.198350e+09,Domestic,4.706104,BZ,Rede D'Or Sao Luiz S.A. provides medical and s...
ZG625748 Corp,ZG625748 Corp,ENGIE BRASIL ENERGIA SA,BBG01KH2VYK0,BREGIEDBS0E7,11/15/2023,2028-11-15,BRL,FIXED,10.9000,#N/A Field Not Applicable,...,DOMESTIC,BR,2028-11-15,AT MATURITY,318122000,3.181220e+08,Domestic,4.037184,FP,Engie provides low-carbon energy and services....
ZH253136 Corp,ZH253136 Corp,ENAUTA PARTICIPACOES,BBG01JLW31Z9,BRENATDBS044,9/15/2023,2029-09-17,BRL,FIXED,13.9662,#N/A Field Not Applicable,...,DOMESTIC,BR,2024-03-15,AT MATURITY,996504000,9.965040e+08,Domestic,6.396052,BZ,Brava Energia operates as a oil and gas compan...


In [8]:

# ────────────────────────────────────────────────────────────────
# 4. Historical YAS yield matrix
# ────────────────────────────────────────────────────────────────
ylds = pd.read_excel(YIELD_PATH, sheet_name="ya_values_only")
ylds.rename(columns={ylds.columns[0]: "OBS_DATE"}, inplace=True)
ylds["OBS_DATE"] = pd.to_datetime(ylds["OBS_DATE"])
ylds = ylds.set_index("OBS_DATE").sort_index()
ylds

,EC744185 Corp,BV932351 Corp,ZN887990 Corp,BX122675 Corp,ZD181681 Corp,ZJ024663 Corp,YN794038 Corp,BV905494 Corp,BQ182881 Corp,YX077422 Corp,...,EJ745835 Corp,ZG074649 Corp,EK062024 Corp,YT660642 Corp,ZD833929 Corp,ZM429069 Corp,BR853537 Corp,BT353913 Corp,BZ862695 Corp,YT877069 Corp
OBS_DATE,,,,,,,,,,,,,,,,,,,,,
1989-10-31,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0,0.0,0.000000,0.000000,0.000000,0
1989-11-30,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0,0.0,0.000000,0.000000,0.000000,0
1989-12-29,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0,0.0,0.000000,0.000000,0.000000,0
1990-01-31,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0,0.0,0.000000,0.000000,0.000000,0
1990-02-28,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0,0.0,0.000000,0.000000,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-28,NaN,-6666.000000,3.148550,3.159477,1.076930,1.306744,0,32.730055,1.255730,0.597712,...,-6666.000000,5.998689,0.000000,12.719988,0,NaN,-7.308758,0.443472,-2.670922,0
2025-03-31,NaN,926.058368,3.103330,3.532190,1.153315,1.294116,0,32.786838,1.172592,2.407520,...,88403.659687,6.162208,0.000000,13.132672,0,NaN,-7.908430,0.324448,-2.464705,0
2025-04-30,NaN,1211.869713,3.580347,3.784295,1.147126,1.306476,0,33.912204,1.192853,2.406513,...,163787.988422,5.615661,-14.691093,12.601764,0,NaN,-8.407286,0.274909,-2.552571,0


In [11]:

# ────────────────────────────────────────────────────────────────
# 5. Helper functions
# ────────────────────────────────────────────────────────────────
def clean_yield_series(y: pd.Series) -> pd.Series:
    """Return decimal yields with 0 % < y < 40 % and finite."""
    y = pd.to_numeric(y, errors="coerce")
    return y.where((y > 0.0) & (y < 0.40))


def build_zero_curve(ref_date: pd.Timestamp, yld_row: pd.Series) -> pd.Series:
    """
    Bootstrap a zero-curve for *ref_date*.
    Returns Series indexed by time-to-maturity (yrs) with DECIMAL rates.
    """
    df = bonds_static.copy()

    df["YAS_BOND_YLD"] = clean_yield_series(df.index.to_series()
                                            .map(yld_row) / 100.0)  # % → decimal
    df = (
        df.dropna(subset=["YAS_BOND_YLD"])
          .loc[df["MATURITY"].dt.date > ref_date.date()]
    )
    if len(df) < 5:
        raise ValueError("≤ 4 clean bonds – skip")

    # split
    zc_df    = df[df["CPN_TYP"] == "ZERO COUPON"].sort_values("MATURITY")
    fixed_df = df[df["CPN_TYP"] == "FIXED"].sort_values("MATURITY")

    zc_prices, zc_cf = [], []
    for mat, y in zip(zc_df["MATURITY"], zc_df["YAS_BOND_YLD"]):
        b = CorpsCalcs1(expiry=mat.date(), rate=y, ref_date=ref_date)
        zc_prices.append(b.price)
        zc_cf.append(pd.Series(index=[mat.date()], data=[b.principal]))

    fixed_prices, fixed_cf = [], []
    for mat, y, cr, fq, fcd in zip(
        fixed_df["MATURITY"],
        fixed_df["YAS_BOND_YLD"],
        fixed_df["CPN"].astype(float) / 100.0,
        fixed_df["CPN_FREQ"].fillna(1).astype(int),
        fixed_df["FIRST_CPN_DT"],
    ):
        b = CorpsCalcs1(
            expiry   = mat.date(),
            rate     = y,
            coupon_rate = cr,
            freq     = fq,
            ref_date = ref_date,
            first_coupon_date = fcd.date() if pd.notna(fcd) else None,
        )
        fixed_prices.append(b.price)
        fixed_cf.append(b.cash_flows)

    cb = CurveBootstrap(
        prices     = zc_prices + fixed_prices,
        cash_flows = zc_cf     + fixed_cf,
        ref_date   = ref_date,
    )

    
    # --- attach zero rates -------------------------------------------------
    curve = cb.zero_curve  # decimal
    curve_times = np.array([
        DAYCOUNT.tf(ref_date, d) if np.issubdtype(curve.index.dtype, np.datetime64)
        else d for d in curve.index
    ], dtype=float)
    curve_rates = curve.values
    if interp1d:
        fz = interp1d(curve_times, curve_rates, kind="linear",
                      bounds_error=False, fill_value="extrapolate")
        get_zr = lambda t: float(fz(t))
    else:
        get_zr = lambda t: float(np.interp(
            t, curve_times, curve_rates,
            left=curve_rates[0], right=curve_rates[-1]
        ))
    df["ZERO_RATE"] = [
        get_zr(DAYCOUNT.tf(ref_date, m)) for m in df["MATURITY"]
    ]

    # --- store snapshot ----------------------------------------------------
    df["OBS_DATE"] = ref_date.date()
    audit_cols = ["id", "OBS_DATE", "MATURITY", "CPN_TYP", "CPN",
                  "YAS_BOND_YLD", "ZERO_RATE"]
    AUDIT_LOG.append(df.reset_index()[audit_cols])

    
    return cb.zero_curve  # decimals




In [12]:

# ────────────────────────────────────────────────────────────────
# 6. Build zero-rate surface
# ────────────────────────────────────────────────────────────────
surface_rows = []
for obs_date, yld_row in ylds.iterrows():
    try:
        yld_row = yld_row.apply(pd.to_numeric, errors="coerce")  # <-- converte para float
        curve = build_zero_curve(obs_date, yld_row)
        ttms  = ([DAYCOUNT.tf(obs_date, d) for d in curve.index]
                 if np.issubdtype(curve.index.dtype, np.datetime64)
                 else curve.index.astype(float))
        surface_rows.append(pd.DataFrame({
            "obs_date":  obs_date,
            "ttm":       ttms,
            "zero_rate": curve.values * 100.0
        }))
    except ValueError as e:
        print(f"[WARNING] {obs_date}: build_zero_curve failed → {e}")

if not surface_rows:
    raise RuntimeError("No curves built – check data/filters")

surface = pd.concat(surface_rows, ignore_index=True)
print(f"Built curves for {surface['obs_date'].nunique()} observation dates.")

# ── consolidate audit snapshots
AUDIT_DF = pd.concat(AUDIT_LOG, ignore_index=True) if AUDIT_LOG else pd.DataFrame()

# ── add DAYS_TO_MATURITY & TENOR_BUCKET
if not AUDIT_DF.empty:
    AUDIT_DF["DAYS_TO_MATURITY"] = (
        pd.to_datetime(AUDIT_DF["MATURITY"]) -
        pd.to_datetime(AUDIT_DF["OBS_DATE"])
    ).dt.days
    AUDIT_DF["DAYS_TO_MATURITY"] = pd.to_numeric(AUDIT_DF["DAYS_TO_MATURITY"], errors='coerce')
    AUDIT_DF["TENOR_YRS"] = AUDIT_DF["DAYS_TO_MATURITY"] / 365.25


    TENOR_BRACKETS = {
        "30-year": 30.0, "10-year": 10.0, "5-year": 5.0, "3-year": 3.0,
        "2-year": 2.0, "1-year": 1.0, "6-month": 0.5,
        "3-month": 0.25, "1-month": 1.0/12.0,
    }
    names = list(TENOR_BRACKETS.keys())
    vals  = np.array(list(TENOR_BRACKETS.values()))

    def bucketize(years: float) -> str:
        return names[np.argmin(np.abs(vals - years))]

    AUDIT_DF["TENOR_BUCKET"] = AUDIT_DF["TENOR_YRS"].apply(bucketize)



[WARNING] 1989-10-31 00:00:00: build_zero_curve failed → ≤ 4 clean bonds – skip
[WARNING] 1989-11-30 00:00:00: build_zero_curve failed → ≤ 4 clean bonds – skip
[WARNING] 1989-12-29 00:00:00: build_zero_curve failed → ≤ 4 clean bonds – skip
[WARNING] 1990-01-31 00:00:00: build_zero_curve failed → ≤ 4 clean bonds – skip
[WARNING] 1990-02-28 00:00:00: build_zero_curve failed → ≤ 4 clean bonds – skip
[WARNING] 1990-03-30 00:00:00: build_zero_curve failed → ≤ 4 clean bonds – skip
[WARNING] 1990-04-30 00:00:00: build_zero_curve failed → ≤ 4 clean bonds – skip
[WARNING] 1990-05-31 00:00:00: build_zero_curve failed → ≤ 4 clean bonds – skip
[WARNING] 1990-06-29 00:00:00: build_zero_curve failed → ≤ 4 clean bonds – skip
[WARNING] 1990-07-31 00:00:00: build_zero_curve failed → ≤ 4 clean bonds – skip
[WARNING] 1990-08-31 00:00:00: build_zero_curve failed → ≤ 4 clean bonds – skip
[WARNING] 1990-09-28 00:00:00: build_zero_curve failed → ≤ 4 clean bonds – skip
[WARNING] 1990-10-31 00:00:00: build_zer

[WARNING] 1991-11-29 00:00:00: build_zero_curve failed → ≤ 4 clean bonds – skip
[WARNING] 1991-12-31 00:00:00: build_zero_curve failed → ≤ 4 clean bonds – skip
[WARNING] 1992-01-31 00:00:00: build_zero_curve failed → ≤ 4 clean bonds – skip
[WARNING] 1992-02-28 00:00:00: build_zero_curve failed → ≤ 4 clean bonds – skip
[WARNING] 1992-03-31 00:00:00: build_zero_curve failed → ≤ 4 clean bonds – skip
[WARNING] 1992-04-30 00:00:00: build_zero_curve failed → ≤ 4 clean bonds – skip
[WARNING] 1992-05-29 00:00:00: build_zero_curve failed → ≤ 4 clean bonds – skip
[WARNING] 1992-06-30 00:00:00: build_zero_curve failed → ≤ 4 clean bonds – skip
[WARNING] 1992-07-31 00:00:00: build_zero_curve failed → ≤ 4 clean bonds – skip
[WARNING] 1992-08-31 00:00:00: build_zero_curve failed → ≤ 4 clean bonds – skip
[WARNING] 1992-09-30 00:00:00: build_zero_curve failed → ≤ 4 clean bonds – skip
[WARNING] 1992-10-30 00:00:00: build_zero_curve failed → ≤ 4 clean bonds – skip
[WARNING] 1992-11-30 00:00:00: build_zer

RuntimeError: No curves built – check data/filters